In [1]:
import os
import sys
import json
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler

from opts import parse_opts
from mean import get_mean, get_std
from spatial_transforms import (
    Compose, Normalize, Scale, CenterCrop, CornerCrop, MultiScaleCornerCrop,
    MultiScaleRandomCrop, RandomHorizontalFlip, ToTensor)
from temporal_transforms import LoopPadding, TemporalRandomCrop
from target_transforms import ClassLabel, VideoID
from target_transforms import Compose as TargetCompose
from dataset import get_training_set, get_validation_set, get_test_set
from utils import Logger
from train import train_epoch
from validation import val_epoch
import test
import collections
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from joblib import dump, load
from sklearn import preprocessing
from scipy import stats
from sklearn.metrics import accuracy_score

from models import *

In [2]:
import easydict
opt = easydict.EasyDict({
    "result_path": 'results2',
    "dataset": 'ucf101-music', # 'ucf101',
    "n_classes": 9, 
    "sample_size": 112,
    "sample_duration": 16,
    "initial_scale": 1.0,
    "n_scales": 5,
    "scale_step": 0.84089641525,
    "train_crop": 'corner',
    "learning_rate": 0.1,
    "momentum": 0.9,
    "dampening": 0.9,
    "weight_decay": 0.001,
    "mean_dataset": 'activitynet',
    "no_mean_norm": False,
    "std_norm": False,
    "nesterov": False,
    "optimizer": 'sgd',
    "lr_patience": 10,
    "batch_size": 16,
    "n_epochs": 2,
    "begin_epoch": 1,
    "n_val_samples": 3,
    "ft_begin_index": 5,
    "scale_in_test": 1.0,
    "crop_position_in_test": 'c',
    "no_softmax_in_test": False,
    "no_cuda": False,
    "n_threads": 4,
    "checkpoint": 2,
    "no_hflip": False,
    "norm_value": 1,
    "model": 'resnet',
    "pretained_model_name": 'resnext-101-kinetics',
    "model_depth": 101,
    "resnet_shortcut": 'B',
    "wide_resnet_k": 2,
    "resnext_cardinality": 32,
    "manual_seed": 1,
    'test_subset': 'test',
})
opt.arch = '{}-{}'.format(opt.model, opt.model_depth)
opt.root_path = '/data/qq/CSCE689/video/'
opt.video_path = opt.root_path + 'UCF-music/'
opt.annotation_path = opt.root_path+'UCF-music-annotation/ucf101_music_with_testing.json'

In [3]:
# use two gpu devices on the server, you can customize it depending on how many available gpu devices you have
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [4]:
from models import resnext

# construct model architecture
model = resnext.resnet101(
            num_classes=opt.n_classes,
            shortcut_type=opt.resnet_shortcut,
            cardinality=opt.resnext_cardinality,
            sample_size=opt.sample_size,
            sample_duration=opt.sample_duration)

model = model.cuda()
# wrap the current model again in nn.DataParallel / or we can just remove the .module keys.
model = nn.DataParallel(model, device_ids=None)

# load best weight (we can also refit the model on the combined train-val dataset, 
# but here we simple load the weight and do the final testing)
pretrain = torch.load('./results1/save_50.pth')
model.load_state_dict(pretrain['state_dict'])


/home/qq/CSCE689/3D-ResNets-PyTorch/models/resnext.py:121: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


<All keys matched successfully>

In [5]:
from datasets.ucf101 import UCF101

mean = get_mean(opt.norm_value, dataset='kinetics')
std = get_std(opt.norm_value)
norm_method = Normalize(mean, [1,1,1])


spatial_transform = Compose([
    Scale(opt.sample_size),
    CornerCrop(opt.sample_size, 'c'),
    ToTensor(opt.norm_value), norm_method
])

temporal_transform = LoopPadding(opt.sample_duration)
target_transform = VideoID() # ClassLabel()



# get test data
test_data = UCF101(
    opt.video_path,
    opt.annotation_path,
    'testing',
    0,
    spatial_transform=spatial_transform,
    temporal_transform=temporal_transform,
    target_transform=target_transform,
    sample_duration=16)


# wrap test data
test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=opt.batch_size,
    shuffle=False,
    num_workers=opt.n_threads,
    pin_memory=False)


dataset loading [0/149]


In [6]:
from testing import final_test
test_results, all_output_buffer = final_test(test_loader, model, opt, test_data.class_names)

test


/home/qq/CSCE689/3D-ResNets-PyTorch/testing.py:44: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs = Variable(inputs, volatile=True)
/home/qq/CSCE689/3D-ResNets-PyTorch/testing.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = F.softmax(outputs)


[1/162]	Time 0.986 (0.986)	Data 0.846 (0.846)	
[2/162]	Time 0.138 (0.562)	Data 0.011 (0.428)	
[3/162]	Time 0.124 (0.416)	Data 0.007 (0.288)	
[4/162]	Time 0.135 (0.346)	Data 0.008 (0.218)	
[5/162]	Time 0.280 (0.333)	Data 0.156 (0.206)	
[6/162]	Time 0.124 (0.298)	Data 0.006 (0.172)	
[7/162]	Time 0.124 (0.273)	Data 0.005 (0.148)	
[8/162]	Time 0.127 (0.255)	Data 0.007 (0.131)	
[9/162]	Time 0.243 (0.253)	Data 0.123 (0.130)	
[10/162]	Time 0.123 (0.240)	Data 0.008 (0.118)	
[11/162]	Time 0.124 (0.230)	Data 0.006 (0.108)	
[12/162]	Time 0.128 (0.221)	Data 0.007 (0.099)	
[13/162]	Time 0.328 (0.230)	Data 0.202 (0.107)	
[14/162]	Time 0.127 (0.222)	Data 0.007 (0.100)	
[15/162]	Time 0.126 (0.216)	Data 0.007 (0.094)	
[16/162]	Time 0.131 (0.211)	Data 0.008 (0.088)	
[17/162]	Time 0.324 (0.217)	Data 0.203 (0.095)	
[18/162]	Time 0.128 (0.212)	Data 0.005 (0.090)	
[19/162]	Time 0.130 (0.208)	Data 0.007 (0.086)	
[20/162]	Time 0.124 (0.204)	Data 0.007 (0.082)	
[21/162]	Time 0.253 (0.206)	Data 0.131 (0.084)	
[

In [7]:
examples = ['v_PlayingGuitar_g05_c03', 
            'v_PlayingViolin_g03_c03', 
            'v_PlayingCello_g07_c05', 
            'v_PlayingFlute_g07_c04',
            'v_PlayingPiano_g01_c02']

### Extract clip duration

In [9]:
avi_path = "/data/qq/CSCE689/video/UCF-101"
clip_duration_dict = {}
real_prediction_dict = {}
import os
from moviepy.editor import VideoFileClip
for tvn in test_results['results']:
    clip = VideoFileClip(os.path.join(avi_path, tvn[2:-8], tvn + ".avi"))
    clip_duration_dict[tvn] = [clip.duration, all_output_buffer[tvn]]
    real_prediction_dict[tvn] = test_results['results'][tvn][0]

In [38]:
# predict
clf = load('./hw5_results/linearsvc.joblib') 

x_test_1 = np.load('./features/resnext101_avgpool_test.npy')
x_test_2 = np.load('./features/resnet50_avgpool_test.npy')
x_test = np.concatenate([x_test_1, x_test_2], axis=1)
y_test = np.load('./features/class_names_ucf_test.npy')
y_pred_test_raw = clf.predict(x_test)
y_pred_test_prob_raw = clf.predict_proba(x_test)

# get ground-truth split
name_to_int = {v:k for k,v in test_data.class_names.items()}

split_idx = []
for idx, y_name in enumerate(y_test):
    if idx == 0 or y_name != y_test[idx-1]:
        split_idx.append(idx)

y_pred_test, y_pred_test_prob, y_pred_test_final = {}, {}, {}
for i, split in enumerate(split_idx):
    if i < len(split_idx) - 1:
        y_pred_test[y_test[split]] = y_pred_test_raw[split:split_idx[i+1]]
        y_pred_test_prob[y_test[split]] = y_pred_test_prob_raw[split:split_idx[i+1]]
        y_pred_test_final[y_test[split]] = np.argmax(np.mean(y_pred_test_prob_raw[split:split_idx[i+1]], axis=0))
        
        

### Generate prediction plot for each video  (all label) -- HW5 ensemble ResNext-101 + ResNet 50 + linearSVC

In [51]:
import matplotlib.pyplot as plt
import numpy as np

for tvn in test_results['results']:
    interval = clip_duration_dict[tvn][0]/len(clip_duration_dict[tvn][1])
    x = np.arange(0, clip_duration_dict[tvn][0], interval) + interval
    y = y_pred_test[tvn] + 1  # np.arange(len(test_data.class_names) + 1)
    x = x[:len(y)]
    my_yticks = [''] + list(test_data.class_names.values()) + ['']
    plt.plot(x, y)
    plt.yticks(np.arange(len(my_yticks) + 1), my_yticks)
    plt.xlabel ('time/sec')
    plt.ylabel ('label')
    plt.title("Ground Truth Label:  " + tvn[2:-8]  + "\n Model Avg. Predict:  " + test_data.class_names[y_pred_test_final[tvn]])
    plt.savefig("./hw5_results/fig_all_label/" + tvn, bbox_inches='tight')
    plt.close()
#     break

### Generate prediction plot for each video (one label) -- HW5 ensemble ResNext-101 + ResNet 50 + linearSVC

In [52]:
import matplotlib.pyplot as plt
import numpy as np


# function to return key for any value 
def get_key(val, my_dict): 
    for key, value in my_dict.items(): 
         if val == value: 
            return key 
    return "key doesn't exist"

for tvn in test_results['results']:
    interval = clip_duration_dict[tvn][0]/len(clip_duration_dict[tvn][1])
    x = np.arange(0, clip_duration_dict[tvn][0], interval) + interval
    idx = get_key(tvn[2:-8], test_data.class_names)
    y = np.array([pred[idx] for pred in y_pred_test_prob[tvn]])  # np.arange(len(test_data.class_names) + 1)    
    x = x[:len(y)]
    plt.plot(x, y)
    plt.xlabel ('time/sec')
    plt.ylabel ('pred score for ground truth label')
    plt.title("Ground Truth Label:  " + tvn[2:-8]  + "\n Model Avg. Predict Score:  " + str(np.mean(y))) # str(real_prediction_dict[tvn]['score'])
    plt.savefig("./hw5_results/fig_one_label/" + tvn, bbox_inches='tight')
    plt.close()
#     break

### Generate json file for each video (one label & all label) -- HW5 ensemble ResNext-101 + ResNet 50 + linearSVC

In [44]:
import numpy as np


# function to return key for any value 
def get_key(val, my_dict): 
    for key, value in my_dict.items(): 
         if val == value: 
            return key 
    return "key doesn't exist"

timeTrueLabel = {}
timeAllLabel = {}
for tvn in test_results['results']:
    interval = clip_duration_dict[tvn][0]/len(clip_duration_dict[tvn][1])
    x = np.arange(0, clip_duration_dict[tvn][0], interval) + interval
    idx = get_key(tvn[2:-8], test_data.class_names)
    y_one_label = np.array([pred[idx] for pred in y_pred_test_prob[tvn]]) 
    y_all_label = y_pred_test[tvn]
    x = x[:len(y_one_label)]
    
    timeTrueLabel[tvn] = {tvn[2:-8]: [[str(time), str(y_one_label[idx])] for idx, time in enumerate(x)]}
    timeAllLabel[tvn] = {tvn[2:-8]: [[str(time), test_data.class_names[y_all_label[idx]]] for idx, time in enumerate(x)]}
    

In [47]:
# with open('./hw5_results/fig_one_label/timeTrueLabel.json', 'w') as fp:
#     json.dump(timeTrueLabel, fp)

# with open('./hw5_results/fig_all_label/timeAllLabel.json', 'w') as fp:
#     json.dump(timeAllLabel, fp)

### Save 5 example json files

In [48]:
import json

with open('./hw5_results/fig_one_label/timeTrueLabel.json', 'r') as fp:
    j_one_label = json.load(fp)

with open('./hw5_results/fig_all_label/timeAllLabel.json', 'r') as fp:
    j_all_label = json.load(fp)

In [49]:
len(j_one_label), len(j_all_label)

(148, 148)

In [50]:
fig_one_label = {}
fig_all_label = {}
for e in examples:
    fig_one_label[e] = j_one_label[e]
    fig_all_label[e] = j_all_label[e]
    
# with open('./hw5_results/fig_one_label/example_one_label.json', 'w') as fp:
#     json.dump(fig_one_label, fp)

# with open('./hw5_results/fig_all_label/example_all_label.json', 'w') as fp:
#     json.dump(fig_all_label, fp)